In [55]:
from IPython.display import display, clear_output

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader


import random
import seaborn as sns
import matplotlib.pyplot as plt

DEVICE = torch.device("cpu")
BATCH_SIZE = 128
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCH_SIZE * 70
N_VALID_EXAMPLES = BATCH_SIZE * 40
EVAL_BATCH_SIZE = 128
SEED = 42
WEIGHT_DECAY = 1e-3
GAMMA = 0.9995

In [56]:
df = pd.read_csv('year_pred.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [57]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

In [58]:
import random

In [59]:
set_random_seed(3407) 
random.randint(0,2)

2

In [79]:
from random import randrange
def active_choice(trial):
    coin = random.randint(0, 2)
    if coin == 2:
        print('ReLU')
        return nn.ReLU()
    elif coin == 1:
        print('ReLU6')
        return nn.ReLU6()
    print('Sigmoid')
    return nn.Sigmoid()

In [80]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 2, 4)
    layers = []

    in_features = 90
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 20, 150)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.BatchNorm1d(num_features=out_features))
        #activation = trial.suggest_categorical("activation", [nn.ReLU(), nn.ReLU6(), nn.Sigmoid()])
        layers.append(active_choice(trial))
        #p = trial.suggest_float("dropout_l{}".format(i), 0, 0.1)
        #layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, 1))

    return nn.Sequential(*layers)

In [81]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        super().__init__() 
        self.X = X
        self.y = y
        self.len = len(X)

    def __len__(self): 
        return self.len

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [82]:
def get_mnist():
    # Load FashionMNIST dataset.
    train_dl = DataLoader(
    MyDataset(X_train, y_train),
    batch_size = 128,
    shuffle = True
    )
    test_dl = DataLoader(
        MyDataset(X_test, y_test),
        batch_size = 300,
        shuffle = False
    )


    return train_dl, test_dl

In [83]:
loss_fn = nn.MSELoss()

In [84]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
  #  np.random.seed(seed)
   # random.seed(seed)

In [85]:
1e-3

0.001

In [86]:
GAMMA = 0.9995 
1e-3 

0.001

In [87]:
def objective(trial):

    # Generate the model.
    set_random_seed(3407) 
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["AdamW"])
    lr = trial.suggest_float("lr", 1e-3, 1e-2)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    print('start'*10)
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data.float())
            loss = loss_fn(output.float(), target.float())
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data.float()).view(-1)
                
                correct += (torch.square((output.float() - target.float())).sum()).item()
        accuracy = (correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)) ** (1/2)
        trial.report(accuracy, epoch)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


In [88]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=300, timeout=1000)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-28 19:54:46,664] A new study created in memory with name: no-name-7e5fc6d5-b6e3-47ad-9962-f9b145aee2b2
/Users/andreyborevskiy/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Sigmoid
ReLU6
Sigmoid
startstartstartstartstartstartstartstartstartstart


[I 2021-10-28 19:54:49,890] Trial 0 finished with value: 1635.6262332819194 and parameters: {'n_layers': 3, 'n_units_l0': 98, 'n_units_l1': 62, 'n_units_l2': 147, 'optimizer': 'AdamW', 'lr': 0.008817099995852687}. Best is trial 0 with value: 1635.6262332819194.
/Users/andreyborevskiy/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


ReLU6
ReLU6
ReLU6
Sigmoid
startstartstartstartstartstartstartstartstartstart


KeyboardInterrupt: 